In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
from pathlib import Path

import torch
from torch_geometric.nn import knn_graph
from torch_geometric import transforms
from torch_geometric.data import Data, DataLoader

from geometric_vector_perceptron import GVP_Network

In [16]:
data_path = Path("../data/synthetic")

In [30]:
cnn = torch.from_numpy(np.load(data_path/"cnn.npy"))
synthetic = torch.from_numpy(np.load(data_path/"synthetic.npy"))
with np.load(data_path/"answers.npz") as data:
    # off_center = torch.from_numpy(data["off_center"])
    perimeter = torch.from_numpy(data["perimeter"])
off_center = torch.from_numpy(np.load(data_path/"ocr.npy"))

In [42]:
synthetic.shape

torch.Size([20000, 2, 100, 3])

In [22]:
num_structs = synthetic.shape[0] # number of protein structures, 20k

In [105]:
# add one-hot vector to the last dimension
is_special = torch.zeros((20000, 2, 100, 1))
is_special[:, 1, :3] = 1 
synthetic = torch.cat([synthetic, is_special], dim=3) # (2000, 2, 100, 3) -> (20000, 2, 100, 4) -- last channel corresponds to is_special

In [112]:
synthetic_transforms = transforms.Compose(
    [
        transforms.KNNGraph(k=10),
        transforms.Cartesian()
    ]
)

data_list = [synthetic_transforms(Data(x=synthetic[n, 1], pos=synthetic[n, 0, :, :3], y=torch.tensor([off_center[n], perimeter[n]]))) for n in range(num_structs)]

In [113]:
data_list[0]

Data(edge_attr=[1000, 3], edge_index=[2, 1000], pos=[100, 3], x=[100, 4], y=[2])

In [114]:
dataloader = DataLoader(data_list, batch_size=32)

In [115]:
next(iter(dataloader))

Batch(batch=[3200], edge_attr=[32000, 3], edge_index=[2, 32000], pos=[3200, 3], ptr=[33], x=[3200, 4], y=[64])